In [ ]:
'''
進階功能介紹

Action
    Postback Action
    Message Action
    URI Action
    Datetime picker action

ImageMap Message

Template Message

啟用Server


'''

In [ ]:
'''

載入設定檔

'''

secretFileContentJson=json.load(open("../line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")

In [ ]:
'''

ImageMap

應用場景

從模型內引用ImageMap模型及其相關參數

ImageMap消息只有Server可以寄給用戶

告知要準備不同解析度之照片，並且名字須以指定數字命名


'''


from linebot.models import (
    MessageEvent, ImagemapSendMessage
)


from linebot.models import (
    ImagemapArea, BaseSize, URIImagemapAction, MessageImagemapAction
)


# 照片地圖的基本建構特徵
imagemap_message = ImagemapSendMessage(
    base_url='https://%s/images/eddie' % server_url,
    alt_text='對話機器人經驗分享',
    base_size=BaseSize(height=1040, width=1040),
    actions=[
        URIImagemapAction(
            link_uri='https://www.youtube.com/watch?v=uauzw00I0wY',
            area=ImagemapArea(
                x=0, y=0, width=520, height=1040
            )
        ),
        MessageImagemapAction(
            text='hello',
            area=ImagemapArea(
                x=520, y=0, width=520, height=1040
            )
        )
    ]
)


In [ ]:
'''

Template Message
    分成四種
        Buttons
        Confirm
        Carousel
        Image Carousel

'''


In [8]:
'''

Button篇


'''

from linebot.models import (
    MessageEvent, TemplateSendMessage 
)

from linebot.models.template import(
    ButtonsTemplate, 
    PostbackTemplateAction,
    MessageTemplateAction,
    URITemplateAction,
    DatetimePickerTemplateAction
)


buttons_template_message = TemplateSendMessage(
    alt_text='Buttons template',
    template=ButtonsTemplate(
        thumbnail_image_url='https://%s/images/janine/janine.jpg'% server_url ,
        title='Menu',
        text='Please select',
        actions=[
            PostbackTemplateAction(
                label='postback',
                text='同時也可以讓用戶發送文字訊息',
                data='action=buy&itemid=1'
            ),
            MessageTemplateAction(
                label='message',
                text='對話機器人實作'
            ),
            URITemplateAction(
                label='uri',
                uri='https://www.youtube.com/watch?v=uauzw00I0wY'
            ),
            DatetimePickerTemplateAction(
                label='datetimepicker',
                data="action=buy&itemid=1",
                mode="datetime",
                initial="2017-12-25t00:00",
                max="2018-01-24t23:59",
                min="2017-12-25t00:00"
            )
        ]
    )
)



In [9]:
'''

Confirm篇

'''

from linebot.models.template import (
    ConfirmTemplate
) 


confirm_template_message = TemplateSendMessage(
    alt_text='Confirm template',
    template=ConfirmTemplate(
        text='Are you sure?',
        actions=[
            PostbackTemplateAction(
                label='postback',
                text='postback text',
                data='action=buy&itemid=1'
            ),
            MessageTemplateAction(
                label='message',
                text='message text'
            )
        ]
    )
)


In [ ]:
'''

Carousel篇

'''

from linebot.models.template import (
    CarouselTemplate, CarouselColumn
)

carousel_template_message = TemplateSendMessage(
    alt_text='Carousel template',
    template=CarouselTemplate(
        columns=[
            CarouselColumn(
                thumbnail_image_url='https://%s/images/janine/janine.jpg' % server_url,
                title='this is menu1',
                text='description1',
                actions=[
                    PostbackTemplateAction(
                        label='postback1',
                        text='postback text1',
                        data='action=buy&itemid=1'
                    ),
                    MessageTemplateAction(
                        label='message1',
                        text='message text1'
                    ),
                    URITemplateAction(
                        label='uri1',
                        uri='https://line.me/R/calls'
                    )
                ]
            ),
            CarouselColumn(
                thumbnail_image_url='https://%s/images/janine/janine2.jpg' % server_url,
                title='this is menu2',
                text='description2',
                actions=[
                    PostbackTemplateAction(
                        label='postback2',
                        text='postback text2',
                        data='action=buy&itemid=2'
                    ),
                    MessageTemplateAction(
                        label='message2',
                        text='message text2'
                    ),
                    URITemplateAction(
                        label='uri2',
                        uri='https://line.me/R/nv/stickerShop'
                    )
                ]
            )
        ]
    )
)

In [ ]:
'''

ImageCarouselTemplate

'''


from linebot.models.template import (
    ImageCarouselTemplate,ImageCarouselColumn
)


image_carousel_message = TemplateSendMessage(
    alt_text='ImageCarousel template',
    template=ImageCarouselTemplate(
        columns=[
            ImageCarouselColumn(
                image_url='https://%s/images/eddie/eddie.jpg' % server_url ,
                action=PostbackTemplateAction(
                    label='postback1',
                    text='你選了eddie',
                    data='action=buy&itemid=1'
                )
            ),
            ImageCarouselColumn(
                image_url='https://%s/images/janine/janine2.jpg' % server_url ,
                action=PostbackTemplateAction(
                    label='postback2',
                    text='你選了janine',
                    data='action=buy&itemid=2'
                )
            )
        ]
    )
)    

In [ ]:
'''

設計一個字典


'''

template_message_dict = {
    "map":imagemap_message,
    "image":image_carousel_message,
    "carousel":carousel_template_message,
    "confirm":confirm_template_message,
    "buttons":buttons_template_message
}


In [ ]:
'''

啟動Server

'''

app = Flask(__name__,static_url_path = "/images" , static_folder = "../images/" )


# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'


# 用戶發出文字消息時， 按條件內容, 回傳照片地圖
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    
    line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text))
    
if __name__ == "__main__":
    app.run(host='0.0.0.0')

In [ ]:
'''

那些沒有在SDK裡面的進階功能

https://developers.line.me/en/docs/messaging-api/using-line-url-scheme/

'''

In [ ]:
'''

Opening the camera and camera roll
Opening the location screen
Sharing your bot account
Opening your bot Timeline and account page
Sending text messages
Opening profile information
Opening common LINE app screens
Opening LINE app settings screens
Opening the Sticker Shop
Opening the Theme Shop
Making phone calls with LINE Out

每一個都是URI，在Line調用這些URI，會啟用特殊功能

'''

In [ ]:
'''

Opening the camera and camera roll

line://nv/camera/

'''


In [ ]:
'''

Sending text messages

line://msg/text/?{text_message}

'''

In [ ]:
'''

Opening common LINE app screens

line://nv/chat
line://nv/addFriends

'''

In [ ]:
'''

Line Out

line://calls

'''